#Importing

In [1]:
from skimage import io, color
import numpy as np
import cv2

In [2]:
from google.colab.patches import cv2_imshow

#choose K uniform centers

for this purpose, first,  we caculate the distance between the initial centers as S.
$
S = \sqrt{ \frac{h*w}{K}}
$

which K is the number of desired segments and h & w are the dimensions of the image.By knowing S, we can sweep the pixels of the image with step S (starting at (S/2, S/2)) and find the initial location of the centers.

In [3]:
def initialize(K, img):
  h, w = img[:, : ,0].shape
  cluster = []
  S = int(np.sqrt(h*w / K))
  i = S / 2
  j = S / 2
  cnt = 0
  while i < h:
    while j < w:
      cluster.append((int(i), int(j)))
      j += S
      cnt += 1
    j = S / 2
    i += S
  return np.array(cluster)


#Gradient calculation

For finding the gradient o each pixel, we need to know the difference of the current pixel with the next one (i+1, j+1), in each channel of the Lab image.

In [4]:
def gradient(lab, i, j):
  h, w = lab[:, :, 0].shape
  if j + 1 >= w:
      j = w - 2
  if i + 1 >= h:
      i = h - 2

  gradient = lab[i + 1, j + 1, 0] - lab[i , j , 0] + \
             lab[i + 1, j + 1, 1] - lab[i , j , 1] + \
             lab[i + 1, j + 1, 2] - lab[i , j , 2]

  return gradient

#Updating members of each cluster

We define the corners of our 2S * 2S window and if each corners passes the values 0 or h, w we set them to 0, h, w respectivly.
Then for each pixel in the window we calculate the distance as:

$
{D}_{lab} = {({L}_i - {L}c)}^2 + {({a}_i - {a}c)}^2 + {({b}_i - {b}c)}^2
$

$
{D}_{xy} = {({x}_i - {x}c)}^2 + {({y}_i - {y}c)}^2
$

At last,  we add these two values (weighted):

$
D = {D}_{lab} + α{D}_{xy}
$

In [12]:
def cluster_update(img, lab, center, K, alpha=0.1):
  h, w = img[:, :, 0].shape
  S = int(np.sqrt(h*w / K))
  res = 1e6 * np.ones((h, w, 2))

  c_count = 0
  for c in center:
    #print(c_count)
    low_I  = c[0] - S
    high_I = c[0] + S
    low_J  = c[1] - S
    high_J = c[1] + S
    if low_I<0:
      low_I = 0
    if high_I > h:
      high_I = h
    if low_J <0:
      low_J = 0
    if high_J > w:
      high_J = w

    for i in range(low_I, high_I):
      for j in range(low_J, high_J):
        D_lab = (lab[i, j, 0] - lab[c[0], c[1], 0])**2 + (lab[i, j, 1] - lab[c[0], c[1], 1])**2 +(lab[i, j, 2] - lab[c[0], c[1], 2])**2
        D_xy  = (i - c[0])**2 + (j - c[1])**2
        D = D_lab + alpha * D_xy
        if D < res[i, j, 0]:
          res[i, j, 0] = D
          res[i, j, 1] = c_count
    c_count += 1

  return res



#Updating the center of each cluster

For updating the center, for each segment we find the pixels related to it (using the res matrix's secod layer : res[:, :, 1], which we saved each pixel's segment in it), then we calculate the mean of x and y of the pixels individually. the new x and y are the new coordinates of the segment's center.

In [6]:
def update_center(c_count, res):
  update_center = []
  for l in range(c_count):
    sum_I = 0
    sum_J = 0
    ind = np.where(res[:, :, 1]==l)
    for i in range(len(ind[0])):
      sum_I += ind[0][i]
      sum_J += ind[1][i]
    new_cI = int(round(sum_I / len(ind[0])))
    new_cJ = int(round(sum_J / len(ind[0])))
    update_center.append((new_cI, new_cJ))

  update_center = np.array(update_center)

  return update_center


#iterations

after initializing, and moving the segment's center based on gradient, we should update the segments and their centers iteratively. we do this in a for loop.

In [8]:
def iteration(cluster0, lab, img, K, alpha):
  center = []
  cnt = 0
  for c0 in cluster0:
    grad_max = 1000
    for i in range(-2, 3):
      for j in range(-2, 3):
        grad = gradient(lab, c0[0]+i, c0[1]+j)
        if grad < grad_max:
          grad_max = grad
          I = c0[0]+i
          J = c0[1]+j

    center.append((I, J))
    cnt += 1

  center = np.array(center)
  c_count = center.shape[0]

  iter = 5
  new_center = center.copy()
  for i in range(iter):
    print(i)
    res = cluster_update(img, lab, new_center, K, alpha)
    new_center = update_center(c_count, res)

  return new_center, res

#running the algorithm

##K=64

In [7]:
img = io.imread('/content/GoldenJellyfish.jpg')
lab = color.rgb2lab(img)
K = 64
cluster0 = initialize(K, img)


In [9]:
img1 = img.copy()
new_center, res = iteration(cluster0, lab, img, K, 0.1)
for c in new_center:
  img1 = cv2.circle(img1, (c[1], c[0]), radius=2, color=(0, 0, 255), thickness=2)

cv2_imshow(img1)

Output hidden; open in https://colab.research.google.com to view.

At last we should plot the borders of each segment. For this purpose, we compare the class of each pixel with its neighbors. it is clear that if one pixel is on the border of one segment, its class is different from some of its neighbors. using this fac we draw a small circle on each pixel which has different clas from at least one of its neighbors.

In [10]:
h, w = img[:, :, 0].shape
img1 = img.copy()
for i in range(1, h-1):
  for j in range(1, w-1):
    up    = res[i-1, j, 1]
    left  = res[i, j-1, 1]
    right = res[i, j+1, 1]
    down  = res[i+1, j, 1]
    upR   = res[i-1, j+1, 1]
    upL   = res[i-1, j-1, 1]
    downR = res[i+1, j+1, 1]
    downL = res[i+1, j-1, 1]
    c     = res[i, j, 1]
    if(c!=up or c!=down or c!=left or c!=right or c!=upR or c!=upL or c!=downR or c!=downL):
      img1 = cv2.circle(img1, (j, i), radius=2, color=(255, 0, 0), thickness=1)



In [11]:
cv2_imshow(img1)

Output hidden; open in https://colab.research.google.com to view.

##K=256

In [13]:
img = io.imread('/content/GoldenJellyfish.jpg')
lab = color.rgb2lab(img)
K = 256
cluster0 = initialize(K, img)

In [14]:
img1 = img.copy()
new_center, res = iteration(cluster0, lab, img, K, 0.1)
for c in new_center:
  img1 = cv2.circle(img1, (c[1], c[0]), radius=2, color=(0, 0, 255), thickness=2)

cv2_imshow(img1)

Output hidden; open in https://colab.research.google.com to view.

In [15]:
h, w = img[:, :, 0].shape
img1 = img.copy()
for i in range(1, h-1):
  for j in range(1, w-1):
    up    = res[i-1, j, 1]
    left  = res[i, j-1, 1]
    right = res[i, j+1, 1]
    down  = res[i+1, j, 1]
    upR   = res[i-1, j+1, 1]
    upL   = res[i-1, j-1, 1]
    downR = res[i+1, j+1, 1]
    downL = res[i+1, j-1, 1]
    c     = res[i, j, 1]
    if(c!=up or c!=down or c!=left or c!=right or c!=upR or c!=upL or c!=downR or c!=downL):
      img1 = cv2.circle(img1, (j, i), radius=2, color=(255, 0, 0), thickness=1)

In [16]:
cv2_imshow(img1)

Output hidden; open in https://colab.research.google.com to view.

##K=1024

In [22]:
img = io.imread('/content/GoldenJellyfish.jpg')
lab = color.rgb2lab(img)
K = 1024
cluster0 = initialize(K, img)

In [23]:
img1 = img.copy()
new_center, res = iteration(cluster0, lab, img, K, 0.1)
for c in new_center:
  img1 = cv2.circle(img1, (c[1], c[0]), radius=2, color=(0, 0, 255), thickness=2)

cv2_imshow(img1)

Output hidden; open in https://colab.research.google.com to view.

In [24]:
h, w = img[:, :, 0].shape
img1 = img.copy()
for i in range(1, h-1):
  for j in range(1, w-1):
    up    = res[i-1, j, 1]
    left  = res[i, j-1, 1]
    right = res[i, j+1, 1]
    down  = res[i+1, j, 1]
    upR   = res[i-1, j+1, 1]
    upL   = res[i-1, j-1, 1]
    downR = res[i+1, j+1, 1]
    downL = res[i+1, j-1, 1]
    c     = res[i, j, 1]
    if(c!=up or c!=down or c!=left or c!=right or c!=upR or c!=upL or c!=downR or c!=downL):
      img1 = cv2.circle(img1, (j, i), radius=2, color=(255, 0, 0), thickness=1)

In [25]:
cv2_imshow(img1)

Output hidden; open in https://colab.research.google.com to view.

##K=2048

In [26]:
img = io.imread('/content/GoldenJellyfish.jpg')
lab = color.rgb2lab(img)
K = 2048
cluster0 = initialize(K, img)

In [27]:
img1 = img.copy()
new_center, res = iteration(cluster0, lab, img, K, 0.1)
for c in new_center:
  img1 = cv2.circle(img1, (c[1], c[0]), radius=2, color=(0, 0, 255), thickness=2)

cv2_imshow(img1)

Output hidden; open in https://colab.research.google.com to view.

In [28]:
h, w = img[:, :, 0].shape
img1 = img.copy()
for i in range(1, h-1):
  for j in range(1, w-1):
    up    = res[i-1, j, 1]
    left  = res[i, j-1, 1]
    right = res[i, j+1, 1]
    down  = res[i+1, j, 1]
    upR   = res[i-1, j+1, 1]
    upL   = res[i-1, j-1, 1]
    downR = res[i+1, j+1, 1]
    downL = res[i+1, j-1, 1]
    c     = res[i, j, 1]
    if(c!=up or c!=down or c!=left or c!=right or c!=upR or c!=upL or c!=downR or c!=downL):
      img1 = cv2.circle(img1, (j, i), radius=2, color=(255, 0, 0), thickness=1)

In [29]:
cv2_imshow(img1)

Output hidden; open in https://colab.research.google.com to view.